# Gather Data
Uses finnhub.io API to gather historical stock data over period of time for items in provided sectors.

In [ ]:
from datetime import datetime

# Select Timeframe
today = datetime.now()

start = datetime(2019, 1, 1, 0, 0, 0)
end = today
# end = datetime(2019, 5, 31, 0, 0, 0)

from src.get_sector_history import get_sector_histories
from src.sectors import Sectors

sector_stock_histories = get_sector_histories(start, end, sectors=[
    "fang"
])

from src.sectors import get_sector_stocks

for sector, history in sector_stock_histories.items():
    print("{}: {}/{}".format(sector, len(history), len(get_sector_stocks(sector))))

## Strategy

In [ ]:
from datetime import timedelta

# yesterday's top gainer
from src.strategy.util.daily import daily
from src.trade import Trade

def use_top_gainer_fixed_purchasing(top_n, target_value, days_to_sell):
    """
    Strategy to buy previous day's top N gainers.
    """

    state = {
        "prev_day": {},
#         "sell": [],
    }

    @daily
    def top_gainer_fixed_purchasing(timestamp, records_by_symbol):

        # Order previous day's stocks by day's gain
        yesterdays_by_gainage = sorted(state["prev_day"].values(), key=lambda d: -d.closing/d.opening)
        # Remove losers
        # yesterdays_by_gainage = list(filter(lambda d: d.closing > d.opening, yesterdays_by_gainage))
        # Get symbols of previous day's stocks ordered by gainage
        symbols_yesterdays_by_gainage = list(map(lambda d: d.symbol, yesterdays_by_gainage))

        # Only consider today's stocks that also traded yesterday
        today_symbol_records = list(records_by_symbol.items())
        today_symbol_records = list(filter(lambda t: t[0] in symbols_yesterdays_by_gainage, today_symbol_records))

        # Order today's stocks by yesterday's order
        todays_days_by_yesterdays_gainage_tuples = sorted(today_symbol_records, key=lambda t: symbols_yesterdays_by_gainage.index(t[0]))
        todays_days_by_yesterdays_gainage = list(map(lambda t: t[1], todays_days_by_yesterdays_gainage_tuples))

        stocks_to_trade = todays_days_by_yesterdays_gainage[:top_n]
        
        trades = list(map(lambda stock_day: Trade(stock_day, target_value // stock_day.closing), stocks_to_trade))
        for trade in trades:
            yield trade

        state["prev_day"] = records_by_symbol
        
#         # SELL
#         today = datetime.strptime(timestamp, "%Y-%m-%d")
#         later = (today + timedelta(days=days_to_sell))

#         state["sell"].extend([(later, t) for t in trades])
        
#         state_trades = []
#         for d, t in state["sell"]:
#             if d > today:
#                 state_trades.append((d, t))  # defer
#                 continue

#             if trade.symbol in records_by_symbol:
#                 day = records_by_symbol[trade.symbol]
#                 yield Trade(day, -trade.quantity)
#             else:
#                 print("UHOH, {} {}".format(timestamp, trade))
        
#         state["sell"] = state_trades
            
    top_gainer_fixed_purchasing.__name__ += "(top_n={top_n}, target_value={target_value}, days_to_sell={days_to_sell})".format(top_n=top_n, target_value=target_value, days_to_sell=days_to_sell)
    return top_gainer_fixed_purchasing
        

In [ ]:
from src.strategy.top_volume import use_top_volume_fixed_purchasing
from src.strategy.buy_preset import use_buy_preset_value_strategy
from src.run_strategies import run_strategies

strategies = [
    use_top_gainer_fixed_purchasing(2, 2000, 5),
#     use_buy_preset_value_strategy(1000),
]

results = run_strategies(strategies, sector_stock_histories, start, end)

In [ ]:
# Evaluate
print("# ROI % EVALUATION ({start} to {end})".format(start=start.strftime("%Y-%m-%d"), end=end.strftime("%Y-%m-%d")))
from src.reporting.roi import print_roi
print_roi(results)


In [ ]:
if today == end:
    print("# TODAY'S TRADES")
    from src.reporting.trades import print_day_trades
    print_day_trades(results, end)


In [ ]:
# Print out trades
for strategy_name, sector_result_map in results.items():
    print("{strategy_name}:".format(strategy_name=strategy_name))

    sector_pairings = list(sector_result_map.items())
    sector_pairings.sort(key=lambda t: t[1]["evaluation"]["period_roi"])

    for sector, sector_results in sector_pairings:
        print("\t{sector}:".format(sector=sector))
        
        with open("./outputs/{sector} - {strategy_name}.csv".format(
            strategy_name=strategy_name.split("(")[0],
            sector=sector,
        ), "w") as phile:
            lines = [
                "timestamp,symbol,quantity,closing,volume\n"
            ]
            for t in sorted(sector_results["trades"], key=lambda t: t["stock_day"]["timestamp"]):
                lines.append("{timestamp},{symbol},{quantity},{closing},{volume}\n".format(**t["stock_day"], quantity=t["quantity"]))
            phile.writelines(lines)
        
        # Summarize
        counts = {}

        for t in sorted(sector_results["trades"], key=lambda t: t["stock_day"]["timestamp"]):
            symbol = t["stock_day"]["symbol"]
            try:
                counts[symbol] += 1
            except:
                counts[symbol] = 1
        
        for ticker, frequency in sorted(counts.items(), key=lambda t: t[1]):
            print("\t\t{}: picked {} times".format(ticker, frequency))
